In [4]:
!pip install pyspark

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [6]:
spark

In [27]:
# Load CSV into Spark DataFrame
df = spark.read.csv("all_reviews_100000_sample.csv", header=True, inferSchema=True)

# Show df
df.show()

+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------+-------------------------------------+-----------------+-----------------+--------+--------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+---------------------+--------------------+
|recommendationid|  appid|                game|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played| language|                               review|timestamp_created|timestamp_updated|voted_up|votes_up|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|hidden_in_steam_china|steam_china_location|
+----------------+-------+--------------------+-----------------+---------------

In [28]:
from pyspark.sql.functions import col

# Convert column to float
df = df.withColumn("weighted_vote_score", col("weighted_vote_score").cast("float"))

# Then filter
df_cleaned = df.filter(col("weighted_vote_score") != 0)
df_cleaned.show()

###Did this approach first but it ruined the whole df not sure why###

# # Make weighted_vote_score non 0
# df_cleaned = df.filter(df.weighted_vote_score != 0)

# # Show the cleaned DataFrame
# df_cleaned.show()

###realized apparently it wasnt already a float so i needed to convert the column first###

+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------+-------------------------------------+-----------------+-----------------+--------+--------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+---------------------+--------------------+
|recommendationid|  appid|                game|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played| language|                               review|timestamp_created|timestamp_updated|voted_up|votes_up|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|hidden_in_steam_china|steam_china_location|
+----------------+-------+--------------------+-----------------+---------------

In [29]:
# Drop multiple columns
df_cleaned = df_cleaned.drop("steam_china_location", "hidden_in_steam_china")

# Show the updated DataFrame
df_cleaned.show()

+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------+-------------------------------------+-----------------+-----------------+--------+--------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+
|recommendationid|  appid|                game|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played| language|                               review|timestamp_created|timestamp_updated|voted_up|votes_up|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|
+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+----

In [30]:
df_cleaned = df_cleaned.withColumn("author_playtime_forever", col("author_playtime_forever").cast("float") / 60)

df_cleaned.show()

+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------+-------------------------------------+-----------------+-----------------+--------+--------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+
|recommendationid|  appid|                game|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played| language|                               review|timestamp_created|timestamp_updated|voted_up|votes_up|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|
+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+----

In [32]:
### Not necessary just thought it would make playtime cleaner ###
from pyspark.sql.functions import round

# Round playtime to whole number
df_cleaned = df_cleaned.withColumn("author_playtime_forever", round(col("author_playtime_forever"), 0))
df_cleaned.show()

+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+-------------------------+------------------+---------+-------------------------------------+-----------------+-----------------+--------+--------+-----------+-------------------+-------------+--------------+-----------------+---------------------------+
|recommendationid|  appid|                game|   author_steamid|author_num_games_owned|author_num_reviews|author_playtime_forever|author_playtime_last_two_weeks|author_playtime_at_review|author_last_played| language|                               review|timestamp_created|timestamp_updated|voted_up|votes_up|votes_funny|weighted_vote_score|comment_count|steam_purchase|received_for_free|written_during_early_access|
+----------------+-------+--------------------+-----------------+----------------------+------------------+-----------------------+------------------------------+----